In [3]:
import rospy, sys
import moveit_commander
from moveit_commander import MoveGroupCommander
from copy import deepcopy

In [4]:
rospy.init_node('URMove')

In [5]:
arm = MoveGroupCommander('manipulator')
arm.set_pose_reference_frame('base_link')
end_effector_link = "peg_link"
arm.allow_replanning(True)
arm.set_goal_position_tolerance(0.001)
arm.set_goal_orientation_tolerance(0.001)
arm.set_end_effector_link(end_effector_link)
print(arm.get_current_pose('peg_link'))
print(arm.get_current_joint_values())

header: 
  seq: 0
  stamp: 
    secs: 5259
    nsecs: 776000000
  frame_id: "world"
pose: 
  position: 
    x: 0.724727743664
    y: -0.0609540192976
    z: 0.286931191486
  orientation: 
    x: -0.0384266365912
    y: -0.998539953184
    z: 0.00133859318891
    w: 0.0379415822569
[3.2409282923301985, -2.1936409971591946, -1.2426770367138236, -1.3516222139410798, -4.7046227148097115, 1.7468832411815534, -0.0001171053032384961]


In [7]:
start_pose = arm.get_current_pose(end_effector_link).pose
print(start_pose)

position: 
  x: 0.724727743664
  y: -0.0609540192976
  z: 0.286931191486
orientation: 
  x: -0.0384266365912
  y: -0.998539953184
  z: 0.00133859318891
  w: 0.0379415822569


In [11]:

def plan_execute(arm, waypoints): 
    fraction = 0.0   #路径规划覆盖率
    maxtries = 100   #最大尝试规划次数
    attempts = 0     #已经尝试规划次数   
    # 尝试规划一条笛卡尔空间下的路径，依次通过所有路点
    while fraction < 1.0 and attempts < maxtries:
        (plan, fraction) = arm.compute_cartesian_path (
                                waypoints,   # waypoint poses，路点列表
                                0.01,        # eef_step，终端步进值
                                0.0,         # jump_threshold，跳跃阈值
                                True)        # avoid_collisions，避障规划  
        attempts += 1    
        if attempts % 10 == 0:
            rospy.loginfo("Still trying after " + str(attempts) + " attempts...")
                        
    if fraction == 1.0:
        rospy.loginfo("Path computed successfully. Moving the arm.")
        arm.execute(plan)
        rospy.loginfo("Path execution complete.")
        return True
    else:
        rospy.loginfo("Path planning failed with only " + str(fraction) + " success after " + str(maxtries) + " attempts.")
        return False 

target_pose = deepcopy(start_pose)
target_pose.position.x = 0.8
target_pose.position.y = 0
target_pose.position.z = 0.3
target_pose.orientation.x = 0
target_pose.orientation.y = -1
target_pose.orientation.z = 0
target_pose.orientation.w = 0
waypoints = []

waypoints.append(target_pose)
plan_execute(arm, waypoints)



[INFO] [1720582947.539741, 5536.941000]: Path computed successfully. Moving the arm.
[INFO] [1720582948.696524, 5538.095000]: Path execution complete.


True

In [14]:
target_pose.position.z -= 0.05
waypoints = []
waypoints.append(target_pose)
plan_execute(arm, waypoints)

[INFO] [1720583051.494116, 5640.755000]: Path computed successfully. Moving the arm.
[INFO] [1720583052.157502, 5641.416000]: Path execution complete.


True

In [15]:
target_pose.position.x += 0.01
waypoints = []
waypoints.append(target_pose)
plan_execute(arm, waypoints)

[INFO] [1720583079.866480, 5668.859000]: Path computed successfully. Moving the arm.
[INFO] [1720583080.237534, 5669.216000]: Path execution complete.


True

In [17]:
type(target_pose)

geometry_msgs.msg._Pose.Pose